# How to work with gurobi pandas to define decision var with multiple sets (index)

## Root folder and read env variables

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## 2. Create gurobi model

In [ ]:
# create model
m = gp.Model('modelo')

### 3. Create decision variables
- Decision variables that are features in ml models
- Decicion variable that is the output in ml models

#### 3.1 define multiple sets

In [ ]:
# define set time
list_set_time = ['t0', 't1', 't2', 't3', 't4', 't5', 't6']
index_set_time = pd.Index(list_set_time)
index_set_time

In [ ]:
# define set additional - second set defined to this example - of example, think the set represent the kind of product
list_set_product = ['w', 'x', 'y', 'z']
index_set_product = pd.Index(list_set_product)
index_set_product

In [ ]:
############## multi set region&time - index pandas ##############
index_set_product_time = pd.MultiIndex.from_product((list_set_product, list_set_time), 
                                                    names = ('product', 'time')
                                                   )
index_set_product_time

#### 3.2 create decision variables MULTIPLE INDEX

In [ ]:
# create decision variables
var_Z1_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable Z1 - MULTI INDEX"
                                     )

var_X2_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable X2 - MULTI INDEX"
                                     )

var_Y2_multiple = gppd.add_vars(m, index_set_product_time, name = "decision variable Y2 - MULTI INDEX"
                                     )

In [ ]:
# "compile"
m.update()

In [ ]:
# see decision var created
var_Y2_multiple

#### 3.3 create decision variables ONE INDEX

In [ ]:
# create decision variables
var_Z1_one = gppd.add_vars(m, index_set_product, name = "decision variable Z1 - ONE INDEX"
                                     )

var_X2_one = gppd.add_vars(m, index_set_product, name = "decision variable X2 - ONE INDEX"
                                     )

var_Y2_one = gppd.add_vars(m, index_set_product, name = "decision variable Y2 - ONE INDEX"
                                     )

In [ ]:
# "compile"
m.update()

In [ ]:
# see decision var created
var_Y2_one

### 5. Define objective optimization
Objetive that no generate infeasibility

In [ ]:
var_Y2_one.sum() # sum across time

In [ ]:
m.setObjective(var_Y2_one.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [ ]:
# solve
m.optimize()

In [ ]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

In [ ]:
# get optimal values and save in a dataframe
######## create a dataframe with set used previosly
solution = pd.DataFrame(index = index_set_product)

######################## save optimal values - features of models (only the features) ########################

# model
solution["var_Z1_one"] = var_Z1_one.gppd.X
solution["var_X2_one"] = var_X2_one.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["var_Y2_one"] = var_Y2_one.gppd.X  # model


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [ ]:
# show value objetive function
opt_objetive_function

In [ ]:
# show value decision variables
solution